<a href="https://colab.research.google.com/github/MaQuest/Summer2021/blob/main/INPUT_TO_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from transformers import BertModel, BertTokenizer
import pandas as pd
import numpy as np
import random
import torch

In [5]:

MODELS = [(BertModel, BertTokenizer, 'bert-base-uncased')]

Using bert-base uncased model


# LOADING OUR BERT MODEL AND TOKENIZER FROM IN-BUILT BERT 


In [6]:
for model_class, tokenizer_class, pretrained_weights in MODELS:
    
    tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
    bert_model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# READING OUR SST-2 SENTIMENT BANK DATA

In [7]:

df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

# USING 4000 SENTENCES FOR FASTER PROCESS

In [8]:

batch = df[:4000]

# TOKENIZING AND PADDING OUR DATA

In [9]:
def tokenize_cut_pad(df):
    
    df = df.copy()
    
    max_input_size = tokenizer.max_model_input_sizes['bert-base-uncased']
    
    # shorten sequences longer than BERT max input size
    df[0] = [text[:max_input_size - 2] for text in df[0].values] 
    tokenized = df[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True))) # tokenizes and converts tokens to ids, includes special tokens
    
    max_len = 0
    for i in tokenized.values:
        if len(i) > max_len:
            # max_len will be equal to longest sequence in the tokenized values
            max_len = len(i)

    padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
    
    return torch.tensor(padded)

# Get BERT model embedding for each CLS token in each example

In [10]:
input_ids = tokenize_cut_pad(batch)


In [11]:
with torch.no_grad():
    last_hidden_states = bert_model(input_ids)[0]

# STORING LAST_HIDDEN_STATE IN VARIABLE FEATURES

In [12]:
features = last_hidden_states[:,0,:].numpy()

# STORING THE TEST SENTIMENT WHETHER 0 OR 1 IN LABELS

In [13]:

labels = batch[1]

In [14]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels,test_size = 0.15)

# DEFAULT SPLIT TO 75-25%

In [15]:

print(train_features.shape)
print(test_features.shape)

(3400, 768)
(600, 768)


# INITIALIZING OUR MODEL

In [16]:

model = LogisticRegression(solver='lbfgs')
model.fit(train_features, train_labels)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

# TESTING OUR MODEL

In [17]:
model.score(test_features, test_labels)

0.8166666666666667

# PREDICTION OF MODEL

# INPUT SENTENCE FROM USER

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/stackexchange_data.csv')

sentences = df.question_text.values[:500]

sentences

In [38]:

def prediction(text):
    
    input_text = tokenizer.encode(text)
    test_input_ids = torch.tensor(input_text)
    test_input_ids = test_input_ids.unsqueeze(0)
    with torch.no_grad():
        hidden_states = bert_model(test_input_ids)[0]
    test_features = hidden_states[:, 0, :].numpy()
    pred = model.predict(test_features)[0]
    return pred

t = 0.0

sentiments = []

score = model.score(test_features, test_labels)
count = 0
for i in sentences:
    pred = prediction(i)
    sentiments.append(i+ " "+ " sentiment score: " + str(pred)+" ")
    print(i+ " "+ " sentiment score: " + str(pred)+" ")

In Microsoft Professional, is there a way for resource task assignments to autopopulate as resource engagements for utilization reporting purposes? If so, how?  sentiment score: 0 
If I have to formulate goals for a project there's the well established S.M.A.R.T. paradigm. It helps to have goals at the end, from which necessary actions can be easily deduced. It prevents from having goals which can be interpreted differently thus causing a lot of problems with stakeholders and sponsors.
I wonder if there are similar paradigms for formulating risks when doing a risk analysis. I often find myself searching for risks by just mindmapping them over every topic which comes to my mind or doing that together with a team. The amount of theoretical situations and constellations posing risks is often overwhelming.
Within an hour of a meeting we wrote down and counted 60 terminal nodes on a mindmap representing some sort of risk in different areas and points of view. I think we are not bad in "risk

RuntimeError: ignored